In [4]:
import numpy as np

def logistic_function(x):
    """
    Computes the logistic (sigmoid) function.
    """
    y = 1 / (1 + np.exp(-x))
    return y


In [5]:
def test_logistic_function():

    x_scalar = 0
    expected_output_scalar = round(1 / (1 + np.exp(0)), 3)
    assert round(logistic_function(x_scalar), 3) == expected_output_scalar

    x_pos = 2
    expected_output_pos = round(1 / (1 + np.exp(-2)), 3)
    assert round(logistic_function(x_pos), 3) == expected_output_pos

    x_neg = -3
    expected_output_neg = round(1 / (1 + np.exp(3)), 3)
    assert round(logistic_function(x_neg), 3) == expected_output_neg

    x_array = np.array([0, 2, -3])
    expected_output_array = np.array([0.5, 0.881, 0.047])
    assert np.all(np.round(logistic_function(x_array), 3) == expected_output_array)

    print("All tests passed!")

test_logistic_function()


All tests passed!


Log Loss Function

In [6]:
def log_loss(y_true, y_pred):
    """
    Computes log loss for binary classification
    """
    y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
    loss = -(y_true * np.log(y_pred)) - ((1 - y_true) * np.log(1 - y_pred))
    return loss


Test Log Loss

In [7]:
def test_log_loss():
    import numpy as np

    assert np.isclose(log_loss(1, 1), 0.0)
    assert np.isclose(log_loss(0, 0), 0.0)

    y_true = 1
    y_pred = 0.8
    expected_loss = -(1 * np.log(0.8))
    assert np.isclose(log_loss(y_true, y_pred), expected_loss)

    y_true = 0
    y_pred = 0.2
    expected_loss = -(1 * np.log(0.8))
    assert np.isclose(log_loss(y_true, y_pred), expected_loss)

    print("All tests passed!")

test_log_loss()


All tests passed!


Cost Function (Average Loss)

In [8]:
def cost_function(y_true, y_pred):
    assert len(y_true) == len(y_pred)

    n = len(y_true)
    loss_vec = log_loss(y_true, y_pred)
    cost = np.sum(loss_vec) / n

    return cost


Test Cost Function

In [11]:

def test_cost_function():
    y_true = np.array([1, 0, 1])
    y_pred = np.array([0.9, 0.1, 0.8])

    expected_cost = (
        (-(1*np.log(0.9)) - (0*np.log(0.1))) +
        (-(0*np.log(0.1)) - (1*np.log(0.9))) +
        (-(1*np.log(0.8)) - (0*np.log(0.2)))
    ) / 3

    result = cost_function(y_true, y_pred)

    assert np.isclose(result, expected_cost, atol=1e-6), f"{result} != {expected_cost}"
    print("Test passed!")

test_cost_function()


Test passed!


Cost Function with Weights (Logistic Regression)

In [12]:
def costfunction_logreg(X, y, w, b):
    n, d = X.shape

    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    cost = cost_function(y, y_pred)
    return cost


Gradient Calculation

In [13]:
def compute_gradient(X, y, w, b):
    n, d = X.shape

    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    grad_w = -(1/n) * np.dot(X.T, (y - y_pred))
    grad_b = -(1/n) * np.sum(y - y_pred)

    return grad_w, grad_b


Gradient Descent (TRAINING)

In [14]:
def gradient_descent(X, y, w, b, alpha, n_iter, show_cost=False, show_params=True):

    cost_history = []
    params_history = []

    for i in range(n_iter):
        grad_w, grad_b = compute_gradient(X, y, w, b)

        w -= alpha * grad_w
        b -= alpha * grad_b

        cost = costfunction_logreg(X, y, w, b)

        cost_history.append(cost)
        params_history.append((w.copy(), b))

        if show_cost and i % 100 == 0:
            print(f"Iteration {i}: Cost = {cost:.6f}")

    return w, b, cost_history, params_history


Prediction Function

In [15]:
def prediction(X, w, b, threshold=0.5):
    z = np.dot(X, w) + b
    y_prob = logistic_function(z)
    y_pred = (y_prob >= threshold).astype(int)
    return y_pred


Evaluation Metrics

In [16]:
def evaluate_classification(y_true, y_pred):

    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    confusion_matrix = np.array([[TN, FP],
                                  [FN, TP]])

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return confusion_matrix, precision, recall, f1_score
